In [ ]:
from  lib.project_functions import * 

# data

In [ ]:
train_ds = tf.data.Dataset.from_generator(
     train_video_data_generation,
     output_signature=(
         tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(1,), dtype=tf.int32)))
train_ds = train_ds.shuffle(100)
train_ds = train_ds.batch(32)

val_ds = tf.data.Dataset.from_generator(
     validation_video_data_generation,
     output_signature=(
         tf.TensorSpec(shape=( None, 224, 224, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(1,), dtype=tf.int32)))
val_ds = val_ds.batch(32)

In [ ]:
train_iterator = iter(train_ds)
val_iterator = iter(val_ds)

print(next(train_iterator)[0].shape, len(next(train_iterator)[1]))
print(next(val_iterator)[0].shape, len(next(val_iterator)[1]))
# (batch_size, time_steps, height, width, channels)

# model

In [ ]:
convnext_tiny  = tf.keras.applications.convnext.ConvNeXtTiny(
                    model_name='convnext_tiny',
                    include_top=False,
                    include_preprocessing=True,
                    weights='imagenet',
                    input_shape=(224, 224, 3))


In [ ]:
# convnext_tiny .summary()

In [ ]:
time_distributed_layer = tf.keras.layers.TimeDistributed(convnext_tiny)

inputs = tf.keras.Input(shape=( None, 224, 224, 3))



time_distributed_output = time_distributed_layer(inputs)
print("Shape of time_distributed_output:", time_distributed_output.shape)

flatten = layers.TimeDistributed(layers.Flatten())(time_distributed_output)
print("Shape of flatten:", flatten.shape)


gru = keras.layers.GRU(128, activation='tanh')(flatten)
dance = layers.Dense(64, activation="relu")(gru)
print("Shape of dance:", dance.shape)

outputs = layers.Dense(5, activation="softmax")(dance)
print("Shape of outputs:", outputs.shape)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

callback = EarlyStopping(
    patience=5,
    monitor="val_loss",
    restore_best_weights=True,
    min_delta=0.001
)

checkpoint = ModelCheckpoint(
    filepath = "modelcheckpoint/video_best_model_1.h5",  
    monitor='val_loss', 
    verbose=1,  
    save_best_only=True,  
    mode='min'  
)


num_train_samples = sum(1 for _ in train_video_data_generation())
steps_per_epoch = round(num_train_samples / 32)


hist = model.fit(train_ds, epochs=50, validation_data=val_ds, verbose=1,
          callbacks=[callback, checkpoint] , steps_per_epoch=steps_per_epoch) 


# ValueError: When providing an infinite dataset, you must specify the number of steps to run (if you did not intend to create an infinite dataset, make sure to not call `repeat()` on the dataset).

In [ ]:
model.save("modelcheckpoint/video_model.png")

In [ ]:
loaded_model = load_model("modelcheckpoint/best_model_1.h5")